In [ ]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
%matplotlib inline
import PIL
import math


In [ ]:
tf. __version__

In [ ]:
def cutWorkSpace(img,border): 
    # dimensioni immagini
    img_h = 2048
    img_l = 2048
    # offset per posizionamento su area d'indagine
    offset_x = 600
    offset_y = 500
    
    #taglio per garantire la ricerca sull'area di interesse, evitando rumori dati dallo sfondo
    crop_img = img[offset_y:img_h-offset_y, offset_x:img_l-offset_x]
    
    # Convert the img to grayscale
    gray = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    high_thresh, thresh_im = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    lowThresh = 0.5*high_thresh
    
    edges = cv2.Canny(blurred,20,150,apertureSize = 3)
    
    # This returns an array of r and theta values
    lines = cv2.HoughLines(edges,1,np.pi/180, 100)
    
    i = 0
    x_first = 0
    x_second = 0
    count = 0
    delta = 35
    thetaFinal = 0
    while count < 2:
        for r,theta in lines[i]:
            # Stores the value of cos(theta) in a
            a = np.cos(theta)

            # Stores the value of sin(theta) in b
            b = np.sin(theta)

            # x0 stores the value rcos(theta)
            x0 = a*r

            # y0 stores the value rsin(theta)
            y0 = b*r

            #ti define line lenght
            const = 3000
            # x1 stores the rounded off value of (rcos(theta)-1000sin(theta))
            x1 = int(x0 + const*(-b))

            # y1 stores the rounded off value of (rsin(theta)+1000cos(theta))
            y1 = int(y0 + const*(a))

            # x2 stores the rounded off value of (rcos(theta)+1000sin(theta))
            x2 = int(x0 - const*(-b))

            # y2 stores the rounded off value of (rsin(theta)-1000cos(theta))
            y2 = int(y0 - const*(a))

            if count == 0 :
                if theta != 0:
                    thetaFinal = 180-math.degrees(theta)
                x_first = x0
                count += 1
            else:
                if abs(x_first-x0) > delta :
                    x_second = x0
                    count += 1
            i += 1
    
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # rotate our image by Theta degrees around the center of the image
    M = cv2.getRotationMatrix2D((cX, cY), -thetaFinal, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h))
    scale_percent = 100 # percent of original size

  
    if x_first < x_second:
        
        median = (x_second+x_first)//2
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)        
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        return  gray
    else:
        median = (x_first+x_second)//2.
        temp = rotated[0:img_h-1, int(offset_x+median-border):int(offset_x+median+border)]        
        #blurred = cv2.GaussianBlur(temp, (3,3), 0)
        width = int(temp.shape[1] * scale_percent / 100)
        height = int(temp.shape[0] * scale_percent / 100)
        dim = (width, height)
        temp = cv2.resize(temp, dim, interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
        return  gray
    

In [ ]:
def divideImage(img,pcs):
    results = []
    lung = len(img) // pcs
    for i in range(pcs):
        results.append(img[(lung*i):(lung*(i+1)-1)][:])
    final_res = []
    for res in results:
        final_res.append(cv2.resize(res, (rows,col), interpolation = cv2.INTER_AREA))
    return final_res

In [ ]:
rows = 32
col = 32

# Estrazione dati di input
Estrazione e preprocessing 

In [ ]:
pathBuoni = '/Users/leonardoaggio/Desktop/Dataset/S606C/Cam3@00004/*.tif'
pathFS = '/Users/leonardoaggio/Desktop/Dataset/S606C/FalsiScarti/*.tif'
pathBuoniVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/BuoniVal/*.tif'
pathFSVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/FalsiScartiVal/*.tif'
pathScartiVal = '/Users/leonardoaggio/Desktop/Dataset/S606C/ScartiVal/*.tif'

rows = 32
col = 32

pezziXimmagine = 17
border = 60

img_train = []
img_test = []


list_paths = []



list_paths = glob.glob(pathBuoni)
list_paths += glob.glob(pathFS) 

count = 0
for img_path in list_paths:
    img = cv2.imread(img_path)
    #try:
    img_train +=  divideImage(cutWorkSpace(img,border),pezziXimmagine)
    count += 1
    #except:
        #print('errore lettura')


list_paths = glob.glob(pathBuoniVal)
list_paths += glob.glob(pathFSVal) 
#list_paths += glob.glob(pathScartiVal) 

count = 0
for img_path in list_paths:
    img = cv2.imread(img_path)
    try:
        img_test +=  divideImage(cutWorkSpace(img,border),pezziXimmagine)
        count += 1
    except:
        print('errore lettura')



#####
# ORA HO IMG_RESULTS & RESULTS
plt.imshow(img_test[0])
plt.show()

In [ ]:

x_train = np.array(img_train)




x_train = x_train/255.
x_temp = np.empty((len(img_train), rows,col,1))
x_train = x_train.astype('float32')
for i in range(len(img_train)):
    x_temp[i]=(np.expand_dims(x_train[i], axis=2))
x_train = x_temp



x_temp = np.empty((len(img_train), rows,col,1))


x_temp = x_train.astype('float32')


x_test = np.array(img_test)
x_test = x_test/255.

x_temp = np.empty((len(img_test), rows,col,1))
for i in range(len(x_test)):
    x_temp[i]=(np.expand_dims(x_test[i], axis=2))
x_test = x_temp


x_train = x_train.astype('float32')

x_test = x_test.astype('float32')


#### Costruzione del modello di autoencoder

In [ ]:
from tensorflow.keras import layers 
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input

inputs = tf.keras.Input(shape=(rows,col, 1), name='input_layer')

encoder = layers.MaxPooling2D((2, 2),padding='same')(inputs)
encoder = layers.Conv2D(16,kernel_size=3,activation='relu',padding='same',strides=1)(encoder)
encoder = layers.MaxPooling2D((2, 2),padding='same')
encoder = layers.Conv2D(32,kernel_size=3,activation='relu',padding='same',strides=1)(encoder)
encoder = layers.MaxPooling2D((2, 2),padding='same')
encoder = layers.Conv2D(32,kernel_size=3,activation='relu',padding='same',strides=1)(encoder)
encoder = layers.Flatten()(encoder)
encoder = layers.Dense(units=512,activation='sigmoid')(encoder)
encoder = layers.Dense(units=256,activation='sigmoid')(encoder)
encoder = layers.Dense(units=256,activation='sigmoid')(encoder)
decoder = layers.Dense(units=512,activation='sigmoid')(encoder)
decoder = layers.Dense(units=512,activation='sigmoid')(dencoder)
decoder = layers.Reshape((4,4,32))(decoder)
decoder = layers.Conv2DTranspose(32,kernel_size=3,activation='relu',padding='same',strides=1)(decoder)
decoder = layers.UpSampling2D((2, 2))
decoder = layers.Conv2DTranspose(16,kernel_size=3,strides=1,activation='relu',padding='same')(decoder)
decoder = layers.UpSampling2D((2, 2))
decoder = layers.Conv2DTranspose(1,kernel_size=3,strides=1,activation='relu',padding='same')(decoder)
decoder = layers.UpSampling2D((2, 2))
outputs = layers.Conv2D(1,kernel_size=(3,3),activation='sigmoid',padding='same')(decoder)




## Definizione funzione di loss e training del modello

In [ ]:
def SSIMLoss(y_true, y_pred):
    
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)

    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))




In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = 0.0005)

from keras.layers import Concatenate, Dense, LSTM, Input, concatenate




In [ ]:
autoencoder = tf.keras.Model(inputs, outputs)
autoencoder.compile(optimizer=optimizer, loss=SSIMLoss)
autoencoder.summary()

In [ ]:
hist=autoencoder.fit(x_train, x_train,
                epochs=150,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test)
                )


In [ ]:
autoencoder.save('newFormatVer1',save_format='h5')

Analisi dei risultati ottenuti selle immagini del train

In [ ]:
decoded_imgs = autoencoder_copy.predict(x_train)
n = 10
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_train[i].reshape(rows,col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    plt.title("Original")
    ax.get_yaxis().set_visible(False)
    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.title("Reconstructed")
    plt.imshow(decoded_imgs[i].reshape(rows,col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
decoded_imgs = autoencoder_copy.predict(x_test)

In [ ]:
values1 = []

for i in range(len(x_test)):
    print(str(i)+'/'+str(len(x_test)))


    value_a = SSIMLoss(x_test[i],decoded_imgs[i])
    values1.append(value_a.numpy())

Calcolo dei risultati relativi a tutte le immagini che rappresentano uno scarto

In [ ]:
path = '/Users/leonardoaggio/Desktop/Dataset/S606C/ScartiTotali/*.tif'
#path = 'Desktop/Dataset/S606C/ScartiTotali/' 
border = 60
#Nclusters = 10

#results = []
imgs = []


list_paths = glob.glob(path) 
count = 0
for img_path in list_paths:
    img = cv2.imread(img_path)
    try:
        plt.imshow(img)
        plt.show()
        temp = cutWorkSpace(img,border)
        imgs +=  divideImage(temp,pezziXimmagine)
        #plt.imshow(temp)
        #plt.show()
        count += 1
    except:
        print('errore lettura')



        
x_test_final = np.array(imgs)
x_test_final = x_test_final/255.       


x_temp = np.empty((len(imgs), rows,col,1))

for i in range(len(imgs)):
    x_temp[i]=(np.expand_dims(x_test_final[i], axis=2))
x_test_final = x_temp


x_test_final = x_test_final.astype('float32')

In [ ]:
decoded_imgs_final = autoencoder_copy.predict(x_test_final)

In [ ]:
values2 = []

for i in range(len(x_test_final)):
    print(str(i)+ ' su '+ str(len(x_test_final)))
    value_a = SSIMLoss(x_test_final[i],decoded_imgs_final[i])
    values2.append(value_a.numpy())

In [ ]:
#decoded_imgs = autoencoder_copy.predict(x_test_final)
n = 10
plt.figure(figsize=(20, 4))

for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test_final[i+10].reshape(rows, col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    plt.title("Original")
    ax.get_yaxis().set_visible(False)
    # Display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.title("Reconstructed")
    plt.imshow(decoded_imgs_final[i+10].reshape(rows, col))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
i = 10
plt.imshow(x_test_final[i].reshape(rows, col))
plt.show()
plt.imshow(decoded_imgs_final[i].reshape(rows, col))
plt.show()

In [ ]:
plt.hist(values1, bins=50)  # arguments are passed to np.histogram
plt.title("Histogram for Model Clf1 Anomaly Scores")
plt.show()

#print(np.min(values1))

plt.hist(values2, bins=50, color='r')  # arguments are passed to np.histogram
plt.title("Histogram for Model Clf1 Anomaly Scores")
plt.show()

print(np.max(values2))

x = values1
y = values2


plt.hist(x, bins=50, alpha=0.5, label='Buoni', color='b')
plt.hist(y, bins=50, alpha=0.5, label='Scarti', color = 'r')
plt.legend(loc='upper right')
plt.show()



In [ ]:

scoreSTX = values2
scoreSTY = []
for i in range(len(scoreSTX)):
    scoreSTY.append((i+1))
    
scoreX = values1
scoreY = []
for i in range(len(scoreX)):
    scoreY.append(i)




Funzione che ritorna la media dei 5 valori maggiori

In [ ]:
def maxFiveAvg(list_in):
    num = 5
    temp = np.copy(list_in)
    ordered = np.sort(temp)
    avg = 0.
    for i in range(num):
        avg += ordered[len(list_in)-1-i]
    avg = avg / num

    return avg
            

In [ ]:
#pezziXimmagine

scoreSTX = []
scoreSTY = []
temp = 0
for i in range(int(len(values2)/pezziXimmagine)):
    scoreSTX.append(maxFiveAvg(values2[i*pezziXimmagine:(i*pezziXimmagine)+pezziXimmagine]))
    scoreSTY.append(i)
    
scoreX = []
scoreY = []
for i in range(int(len(values1)/pezziXimmagine)):
    scoreX.append(maxFiveAvg(values1[i*pezziXimmagine:(i*pezziXimmagine)+pezziXimmagine]))
    scoreY.append(i)




Plot risultati ottenuti

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

plt.scatter(scoreY,scoreX,label='Train Buoni',color='b')
plt.scatter(scoreSTY,scoreSTX,label='Test Scarti',color='r')

plt.legend()

plt.ylabel('Average log-likelihood')
plt.title('PCA score')
plt.savefig('LogLik.png')
plt.show()

Test con threshold manuale

In [ ]:
threshold = 0.16

falsi_scarti = 0
for el in scoreX:
    if el > threshold:
        falsi_scarti += 1
        
scarti = 0
for el in scoreSTX:
    if el > threshold:
        scarti += 1

Test finale su dataset completo 

In [ ]:
path = '/Users/leonardoaggio/Desktop/Dataset/Dataset/Cam3@00004v2/*.tif'
#path = 'Desktop/Dataset/S606C/ScartiTotali/' 
border = 60
#Nclusters = 10

#results = []
imgs = []


list_paths = glob.glob(path) 
count = 0
for img_path in list_paths:
    img = cv2.imread(img_path)
    try:
        temp = cutWorkSpace(img,border)
        imgs +=  divideImage(temp,pezziXimmagine)
        #plt.imshow(temp)
        #plt.show()
        count += 1
    except:
        print('errore lettura')



        
x_test_final_2 = np.array(imgs)
x_test_final_2 = x_test_final_2/255.       


x_temp = np.empty((len(imgs), rows,col,1))

for i in range(len(imgs)):
    x_temp[i]=(np.expand_dims(x_test_final_2[i], axis=2))
x_test_final_2 = x_temp


x_test_final_2 = x_test_final_2.astype('float32')

decoded_imgs_final_2 = autoencoder_copy.predict(x_test_final_2)

values3 = []

for i in range(len(x_test_final_2)):
    print(str(i)+ ' su '+ str(len(x_test_final_2)))
    value_a = SSIMLoss(x_test_final_2[i],decoded_imgs_final_2[i])
    values3.append(value_a.numpy())
    





In [ ]:
scoreSTX_2 = []
scoreSTY_2 = []



In [ ]:
scoreSTX_2 = values3
scoreSTY_2 = []
for i in range(len(scoreSTX_2)):
    scoreSTY_2.append((i+1))



In [ ]:
temp = 0
scoreSTX_2 = []
scoreSTY_2 = []
for i in range(int(len(values3)/pezziXimmagine)):
    scoreSTX_2.append(maxFiveAvg(values3[i*pezziXimmagine:(i*pezziXimmagine)+pezziXimmagine]))
    scoreSTY_2.append(i)

In [ ]:
falsi_scarti = 0
for el in scoreSTX_2:
    if el > threshold:
        falsi_scarti += 1
print(falsi_scarti)

In [ ]:
len(list_paths)

Plot finale

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
plt.scatter(scoreY,scoreX,label='Validation Buoni',color='b')
plt.scatter(scoreSTY,scoreSTX,label='Test Scarti',color='r')
plt.scatter(scoreSTY_2,scoreSTX_2,label='Test Buoni 2',color='g')

plt.plot([-5,500],[threshold,threshold],label=('Limite identificazione scarto'))

plt.legend()

plt.ylabel('Average log-likelihood')
plt.title('PCA score')
plt.savefig('LogLik.png')
plt.show()